In [1]:
import pandas as pd
import sqlite3
import importlib
import numpy as np
from sklearn.linear_model import LinearRegression

# keeping company information in additional file
import data_file

In [ ]:
importlib.reload(data_file)

In [2]:
key_bu = data_file.key_bu

In [ ]:
# preparation of db from excel source copied from Tableau # APPLY ONLY IF NO DB EXISTS

source_file = 'data_files/source_data.xlsx'
# Convert the sheet to a Pandas dataframe
df_1 = pd.read_excel(source_file, sheet_name='1719')
df_2 = pd.read_excel(source_file, sheet_name='2023')

df = pd.concat([df_1, df_2])

conn = sqlite3.connect('data_files/ms_orders_data.db')
df.to_sql('codes', conn, index=False)
conn.close()

In [6]:
# reading database data and transforming it to pandas datafranme

product_orders_db_path = data_file.product_orders_db_path 

conn = sqlite3.connect(product_orders_db_path)
query = "SELECT * FROM codes" 
df = pd.read_sql_query(query, conn)
conn.close()

customer_db_path = data_file.customer_db_path

conn = sqlite3.connect(customer_db_path)
query = "SELECT * FROM customers"  
df_customers = pd.read_sql_query(query, conn)
conn.close()

In [7]:
# cleaning initial data from intercompany orders

df['sold_to_customer_n_latest'] = df['sold_to_customer_n_latest'].astype(str)
#exclusion lines with some specific companies
my_company_name = data_file.my_company_name
exclusion_company_one = data_file.exclusion_company_one
exclusion_company_two = data_file.exclusion_company_two
exclusion_company_three = data_file.exclusion_company_three

#necessary modifications of columns
df['bu'] = df['bu'].astype(str)
df['company_upper'] = df['sold_to_customer_n_latest'].str.upper()
# Filter out rows with unwanted value in column2
df = df[~df['company_upper'].str.contains(my_company_name) & ~df['company_upper'].str.contains(exclusion_company_one) & ~df['company_upper'].str.contains(exclusion_company_two) & ~df['company_upper'].str.contains(exclusion_company_three)] 

In [8]:
# Filter the dataframe based on conditions  delete rows where both columns = 0
df = df[(df['order_intake_quantity'] != 0) | (df['order_intake_euro'] != 0)]

In [9]:
# make the same format for sold_to_customer column for further merging
df['sold_to_customer'] = df['sold_to_customer'].astype(str)
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)

#keep only necessary columns
df_cusmoters_for_merging = df_customers.loc[:, ['sold_to_customer','customer_name', 'indirect_direct', 'channel', 'type', 'tier_new', 'tier']]

df_merged = df.merge(df_cusmoters_for_merging, on='sold_to_customer', how='left')

df_merged = df_merged.loc[:, ['year_month', 'company_code_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n_latest', 'bu', 'bu_n', 'material', 'material_n',
       'ms_code', 'order_intake_quantity', 'order_intake_euro', 'customer_name', 'indirect_direct', 'channel', 'type',
       'tier_new', 'tier']]

In [10]:
# saving information only for key bu groups

df = df_merged[df_merged['bu'].isin(key_bu)]

In [ ]:
# saving information only for necessary channel  - APPLY IT ONLY IF NECESSARY

df = df_merged[df_merged['tier'] == 'Channel Partner']

In [11]:
# adding fiscal year information

temp_df = df.copy()

# Parse 'year_month' column and create 'FY' column. FY start from 4 month
temp_df['year_month_date'] = pd.to_datetime(temp_df['year_month'], format='%Y%m')
temp_df['FY'] = np.where(temp_df['year_month_date'].dt.month >= 4, temp_df['year_month_date'].dt.year, temp_df['year_month_date'].dt.year - 1)

# Create fiscal 'quarter' column
quarter_dict = {1: 'Q4', 2: 'Q4', 3: 'Q4', 4: 'Q1', 5: 'Q1', 6: 'Q1', 7: 'Q2', 8: 'Q2', 9: 'Q2', 10: 'Q3', 11: 'Q3', 12: 'Q3'}
temp_df['quarter'] = temp_df['year_month_date'].dt.month.map(quarter_dict)

# Create 'half_year' column
temp_df['half_year'] = np.where(temp_df['year_month_date'].dt.month.between(4, 9), 'HY1', 'HY2')

In [12]:
# there is some position with added _000000 suffix (stockable position) and auxililary position with suffixes like BOP, ENT, EXP
# it necessary to clean them

# Splitting the 'material' column and keeping the part before '_0'
temp_df['material'] = temp_df['material'].str.split('_0', expand=True)[0]

# Delete rows where 'ms_code' contains text with 'BOP'
temp_df['ms_code'] = temp_df['ms_code'].astype(str)
temp_df = temp_df[~temp_df['ms_code'].str.contains('BOP')]

# Delete rows where 'ms_code' contains text with 'ENT'
temp_df = temp_df[~temp_df['ms_code'].str.contains('ENT')]

# Delete rows where 'ms_code' contains text with 'EXP'
temp_df = temp_df[~temp_df['ms_code'].str.contains('EXP')]

In [ ]:
# keeping only the most valuable position

threshold_value = 0.95

key_materials = temp_df.copy()

# Calculate the sum of orders for each material within each business unit
summary = key_materials.groupby(['bu', 'material'])['order_intake_euro'].sum().reset_index()

# Sort the materials within each business unit based on their sum values in descending order
summary = summary.sort_values(['bu', 'order_intake_euro'], ascending=[True, False])

# Calculate the cumulative sum of the sorted materials within each business unit
summary['cumulative_sum'] = summary.groupby('bu')['order_intake_euro'].cumsum()

# Calculate the threshold value that represents 95% of the total sum value within each business unit
summary['total_sum'] = summary.groupby('bu')['order_intake_euro'].transform('sum')
summary['threshold'] = summary['total_sum'] * threshold_value

# Filter the materials within each business unit based on the threshold value
summary_filtered = summary[summary['cumulative_sum'] <= summary['threshold']]

more_valuable_materials = list(summary_filtered['material'].unique())

key_materials = temp_df[temp_df['material'].isin(more_valuable_materials)]

In [ ]:
key_materials = key_materials.loc[:, ['FY', 'customer_name', 'bu', 'material', 'ms_code', 'order_intake_quantity', 'order_intake_euro']]

# FY23 has just started, so it is better to exlude values for further analysis
key_materials = key_materials[key_materials['FY'] < 2023]

In [ ]:
# products - full code - quantities in Year periods
pivot_df_material_qty = key_materials.pivot_table(index=['customer_name', 'bu', 'material'], 
                          columns='FY', 
                          values='order_intake_quantity', 
                          aggfunc='sum',fill_value=0)

pivot_df_material_qty.reset_index(inplace=True)

In [ ]:
# products quantities in Year periods
pivot_df_material_qty_no_customer = key_materials.pivot_table(index=['bu', 'material'], 
                          columns='FY', 
                          values='order_intake_quantity', 
                          aggfunc='sum',fill_value=0)

pivot_df_material_qty_no_customer.reset_index(inplace=True)

In [ ]:
# VERSION with customer information

# products codes quantities in Year periods
pivot_df_qty = key_materials(index=['customer_name', 'bu', 'material', 'ms_code'], 
                          columns='FY', 
                          values='order_intake_quantity', 
                          aggfunc='sum')

# products codes orders sum in Year periods
pivot_df_sum = key_materials.pivot_table(index=['customer_name', 'bu', 'material', 'ms_code'], 
                          columns='FY', 
                          values='order_intake_euro', 
                          aggfunc='sum')

result_pivot = pivot_df_qty.join(pivot_df_sum, lsuffix='_qty', rsuffix='_eur')


# Resetting the index to make customer_name and material as normal columns
result_pivot.reset_index(inplace=True)

# Renaming the columns for clarity
result_pivot.columns.name = None


for year in years:
    qty_column = f'{year}_qty'
    eur_column = f'{year}_eur'
    
    result_pivot[eur_column] = result_pivot[eur_column] / result_pivot[qty_column]
    result_pivot[eur_column] = result_pivot[eur_column].round(2)
    result_pivot[eur_column] = result_pivot[eur_column].replace([np.inf, -np.inf], 0)
    
#result_pivot = result_pivot.loc[:, ['customer_name', 'bu', 'material', 'ms_code','2017_eur',
#       '2018_eur', '2019_eur', '2020_eur', '2021_eur', '2022_eur', '2023_eur']]
    
#pivot_df_material_qty = pivot_df_material_qty.reset_index()
result_pivot.to_excel('data_files/result_pivot_with_customers.xlsx', index=True)

In [ ]:
# Trends info with customer information

# convert the dataframe to long format
df_long_qty = pivot_df_material_qty.melt(id_vars=['customer_name', 'bu', 'material'], var_name='year', value_name='qty')

# convert year to integer
df_long_qty['year'] = df_long_qty['year'].astype(int)

# Initialize a dictionary to store the results
results = {}

# Group by customer_name and material and run a regression for each group
for (customer_name, material), group in df_long_qty.groupby(['customer_name', 'material']):
    # Skip this group if it only has one row
    if len(group) <= 1:
        continue
        
    group['qty'] = group['qty'].fillna(0)
    
    # Check if group is empty
    if group.empty:
        continue
    

    # Run the regression
    X = group['year'].values.reshape(-1,1)  # reshape is needed because we have only one feature
    y = group['qty']
    model = LinearRegression()
    model.fit(X, y)
    
    # Save the slope coefficient (i.e., trend) in the results
    results[(customer_name, material)] = model.coef_[0]

# Convert the results to a pandas DataFrame
trends = pd.DataFrame.from_dict(results, orient='index', columns=['trend'])

# Reset the index
trends.reset_index(inplace=True)

# Split the tuple into two separate columns
trends[['customer_name', 'material']] = pd.DataFrame(trends['index'].tolist(), index=trends.index)

# Drop the original 'index' column
trends.drop(columns=['index'], inplace=True)

# Set the column names (optional, the column names should be correct now)
# trends.columns = ['customer_name', 'material', 'trend']

# Sort the DataFrame by customer_name and material
trends = trends.sort_values(['customer_name', 'material'])

#pivot_to_excel
trends.to_excel('data_files/trends_with_customers.xlsx', index=True)

In [ ]:
# Trends info without customers inforamtion

# convert the dataframe to long format
df_long_qty = pivot_df_material_qty.melt(id_vars=['customer_name', 'bu', 'material'], var_name='year', value_name='qty')

# convert year to integer
df_long_qty['year'] = df_long_qty['year'].astype(int)

# Initialize a dictionary to store the results
results = {}

# Group by customer_name and material and run a regression for each group
for (material, bu), group in df_long_qty.groupby(['material', 'bu']):
    # Skip this group if it only has one row
    if len(group) <= 1:
        continue
        
    group['qty'] = group['qty'].fillna(0)
    
    # Check if group is empty
    if group.empty:
        continue
    

    # Run the regression
    X = group['year'].values.reshape(-1,1)  # reshape is needed because we have only one feature
    y = group['qty']
    model = LinearRegression()
    model.fit(X, y)
    
    print(model.coef_)
    
    # Save the slope coefficient (i.e., trend) in the results
    results[(material, bu)] = model.coef_[0]

# Convert the results to a pandas DataFrame
trends = pd.DataFrame.from_dict(results, orient='index', columns=['trend'])

# Reset the index
trends.reset_index(inplace=True)

# Split the tuple into two separate columns
trends[['material', 'bu']] = pd.DataFrame(trends['index'].tolist(), index=trends.index)

# Drop the original 'index' column
trends.drop(columns=['index'], inplace=True)

# Set the column names (optional, the column names should be correct now)
# trends.columns = ['customer_name', 'material', 'trend']

# Sort the DataFrame by customer_name and material
trends = trends.sort_values(['material'])

#pivot_to_excel
trends.to_excel('data_files/trends_wo_customers.xlsx', index=True)

In [ ]:
# with forecasting

Convert the dataframe to long format
df_long_qty = pivot_df_material_qty_no_customer.melt(id_vars=['bu', 'material'], var_name='year', value_name='qty')

# Convert year to integer
df_long_qty['year'] = df_long_qty['year'].astype(int)

# Initialize a dictionary to store the results
results = {}

# Group by bu and material and run a regression for each group
for (bu, material), group in df_long_qty.groupby(['bu', 'material']):
    # Skip this group if it only has one row
    if len(group) <= 1:
        continue
    
    group['qty'] = group['qty'].fillna(0)
    
    # Check if group is empty
    if group.empty:
        continue
    
    # Run the regression
    X = group['year'].values.reshape(-1, 1)  # reshape is needed because we have only one feature
    y = group['qty']
    model = LinearRegression()
    model.fit(X, y)
    
    # Save the regression model in the results
    results[(bu, material)] = model

# Create a new dataframe for the forecast
forecast = pd.DataFrame(columns=['bu', 'material', 'year', 'qty'])

# Generate the forecast for each group
for (bu, material), model in results.items():
    # Get the last observed year
    last_year = df_long_qty[(df_long_qty['bu'] == bu) & (df_long_qty['material'] == material)]['year'].max()
    
    # Generate the forecast for 2023 and 2024
    forecast_year = pd.DataFrame({'year': [2023, 2024]})
    forecast_year['bu'] = bu
    forecast_year['material'] = material
    
    # Predict the values for the forecast years
    forecast_year['qty'] = model.predict(forecast_year['year'].values.reshape(-1, 1))
    
    # Append the forecast to the main dataframe
    forecast = forecast.append(forecast_year)

# Sort the forecast dataframe by bu and material
forecast.sort_values(['bu', 'material'], inplace=True)

# Reset the index of the forecast dataframe
forecast.reset_index(drop=True, inplace=True)

#pivot_df_material_qty = pivot_df_material_qty.reset_index()
forecast.to_excel('data_files/forecast_info.xlsx', index=True)